Evan Callia 1-18

Problem 1:
For each county in Washington, create a GeoJSON file whose filename is the county name and whose contents are the polygons for the block groups within that county. This means that you will end up with as many GeoJSON files as there are counties in WA. Each such file should be named after a county and contain only the block groups from that county.

Problem 2:
Print out a ranked list, in descending order, of the ten largest total populations in 2013 of counties in Washington, according to this dataset

Approach: The jsonOutput method takes 7 perameters which allow for the user to specify which files to use, and the names of the fields to use for each file. The approach is to first creat the geojson files. To do this all records of a particular county were selected using a sql expression. From there a new shapefile was created for each county containing all the blocks for that county. The OSGeo4W was then called to convert these shapefiles to geojson files. The approach for ordering the top 10 counties in descending order is to first create a total population field. Then go through the shapefiles for each county (created above) and calculate the total population. The population was then added to the countynames file and sorted. From there the top 10 were printed.

In [17]:
#setting up and importing arcpy
import sys
sys.path.append('C:\\Program Files (x86)\\ArcGIS\\Desktop10.3\\bin')
sys.path.append('C:\\Program Files (x86)\\ArcGIS\\Desktop10.3\\arcpy')
sys.path.append('C:\\Program Files (x86)\\ArcGIS\\Desktop10.3\\ArcToolbox\\Scripts')
import arcpy
arcpy.env.workspace = r"V:\UW\Geog 458\Lab01"
arcpy.env.overwriteOutput = True

#setting up and importing geojson file exporter
from subprocess import call
import os
os.environ["GDAL_DATA"] = "C:\\OSGeo4W\\share\\gdal"
os.environ["GDAL_DRIVER_PATH"] = "C:\\OSGeo4W\\bin\\gdalplugins"
os.environ["PROJ_LIB"] = "C:\\OSGeo4W\\share\\proj"
os.environ["PATH"] = "C:\\OSGeo4W\\bin;"+os.environ["PATH"]+";C:\\OSGeo4W\\apps\\msys\\bin;C:\\OSGeo4W\\apps\\Python27\\Scripts"
    

    
#table1 = r"saep_bg10\saep_bg10.shp" - List of blocks with Coutnty FIPS
#field1 = "COUNTYFIP10"
#table2 = "WashingtonFIPS.dbf" - List of counties with FIPS
#field2 = "FIPSCounty"
#field2_1 = "CountyName"
#out_path = r"Z:\Google Drive\UW\Geog 458\Lab01\Output"
#year = 2013
def jsonOutput(table1, field1, table2, field2, field2_1, out_path, year):
    
    #add totalpop field for each county - to be used when printing Counties in descending order
    arcpy.AddField_management("V:\\UW\\Geog 458\\Lab01\\" + table2, "TotalPop", "DOUBLE")
    
    #make geojson files
    with arcpy.da.SearchCursor(table2, [field2, field2_1]) as cursor:
        for row in cursor:
            
            #out_name = county name
            out_name = row[1]
            
            #select bloks with same county FIPS
            sql = arcpy.AddFieldDelimiters(table1, field1) + " = '" + str(row[0]) + "'"
            
            #make teoporary shapefile for each county containing all blocks 
            arcpy.FeatureClassToFeatureClass_conversion(table1, out_path, out_name, sql)  
            
            #convert county shapefile to geojson file               
            call(['C:\\OSGeo4W\\bin\\ogr2ogr','-f','GeoJSON','-t_srs','WGS84','-s_srs','EPSG:2927','V:\\UW\\Geog 458\\Lab01\\Output\\' + out_name + '.geojson'  ,'V:\\UW\Geog 458\\Lab01\\Output\\' + out_name + ".shp"])
            
            #Print Counties by totalpop Descending
            #populate totalpop field
            with arcpy.da.SearchCursor("V:\\UW\\Geog 458\\Lab01\\Output\\" + out_name + ".shp", ["POP" + str(year)]) as cursor2:
                
                #initialize totalpop to keep track of pop as records are traversed
                totalPop = 0.0
                
                #loop through rows and add total pop
                for row2 in cursor2:
                    totalPop = totalPop + row2[0]
                
                #concatinate sql expression in order to grab proper county record
                sql2 = arcpy.AddFieldDelimiters(table2, field2_1) + " = '" + out_name + "'"
                
                cursor3 = arcpy.da.UpdateCursor(table2, ["TotalPop"], sql2)
                for row3 in cursor3:
                    row3[0] = totalPop
                    cursor3.updateRow(row3)
                    del row3
                del cursor3
                
            #delete individual county shapefile (it was only temporary to make the geojason file and count totalpop)
            arcpy.Delete_management(out_name + ".shp")
            
    #sort totatpol field
    arcpy.Sort_management(table2, "Counties_Sort", [["TotalPop", "DESCENDING"]])
    
    #print top 10 counties
    cursor4 = arcpy.da.SearchCursor("Counties_Sort", ["TotalPop", field2_1])
    print "Top 10 Counties Ordered by Total Poplation in Descending Order:"
    
    #initialize count to keep track of top 10
    count = 0
    
    for row4 in cursor4:
        if count < 10:
            print str(count + 1) + ".\t" + row4[1] + ": " + str(row4[0])
            count += 1
        del row4
    del cursor4
   
    

In [18]:
#table1 = r"saep_bg10\saep_bg10.shp" - List of blocks with Coutnty FIPS
#field1 = "COUNTYFIP10"
#table2 = "WashingtonFIPS.dbf" - List of counties with FIPS
#field2 = "FIPSCounty"
#field2_1 = "CountyName"
#out_path = r"Z:\Google Drive\UW\Geog 458\Lab01\Output"
#year = 2013
jsonOutput(r"saep_bg10\saep_bg10.shp", "COUNTYFP10", 
           "WashingtonFIPS.dbf", "FIPSCounty", "CountyName", 
           r"Z:\Google Drive\UW\Geog 458\Lab01\Output", 2013)

Top 10 Counties Ordered by Total Poplation in Descending Order:
1.	King: 1981900.0
2.	Pierce: 814500.0
3.	Snohomish: 730500.0
4.	Spokane: 480000.0
5.	Clark: 435500.0
6.	Thurston: 260100.0
7.	Kitsap: 254000.0
8.	Yakima: 247250.0
9.	Whatcom: 205800.0
10.	Benton: 183400.0
